In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-04-09


Found rundate 2023-04-08 10:00:02 < 2023-04-09 at commit b36cebf0; returning commit 7e9f4fb


'2023-04-09 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-04-09 10:00:02, NJSP was reporting 125 YTD deaths
Current YTD Deaths (2024-04-09 10:00:02-04:00): 155
Previous year YTD Deaths (adjusted; 2023-04-09 10:00:02): 125.92564596174735
Projected 2024 total: 668.3578512039431
23.1% change


FAUQStats cache miss: 2023, Sun Apr 09 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,STREET,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,NaN,Mulberry St,1.0,0.0,0.0,1.0,0.0,Mulberry St,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,322,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,NaN,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,South Randolphville Rd,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,NaN,Outwater Ln,1.0,0.0,0.0,1.0,0.0,Outwater Ln,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,NaN,Washington St,1.0,0.0,0.0,1.0,0.0,Washington St,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12389,06,Cumberland,0610,Millville City,670,County 670,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-03-31 21:07:00-04:00
12392,20,Union,2013,Rahway City,NaN,West Grand Ave,1.0,0.0,0.0,1.0,0.0,West Grand Ave,NaN,2023-04-01 19:07:00-04:00
12390,07,Essex,0714,Newark City,1,State Highway 1 MP 49.6,2.0,1.0,1.0,0.0,0.0,NaN,NaN,2023-04-02 02:38:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,5,1,5,1,12
Bergen,6,1,7,0,14
Burlington,8,2,2,0,12
Camden,5,1,5,0,11
Cape May,1,0,0,0,1
Cumberland,2,0,1,0,3
Essex,1,2,5,0,8
Gloucester,5,2,0,0,7
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

27.2% through the year, 72.8% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                                 
Atlantic   driver            4       17        5
Bergen     driver            0       20        6
Burlington driver            1       26        8
Camden     driver            3       18        5
Cape May   driver            2        4        1
...                        ...      ...      ...
Sussex     driver            0        6        1
           passenger         0        2        0
           pedestrian        0        1        0
           cyclist           0        0        0
           crashes           0        6        1

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,3,19,7,13
Bergen,53,1,25,5,13
Burlington,68,1,81,6,5
Camden,53,4,23,5,26
Cape May,5,0,3,1,1
Cumberland,16,0,11,3,4
Essex,44,1,14,5,26
Gloucester,34,1,26,7,2
Hudson,26,1,11,6,7


In [9]:
projected.to_csv(PROJECTED_CSV)